**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

<h1>Deploy a Re-ranking model using OCI Data Science, Model Deployment</h1>

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**Prerequisites to run the notebook:**
- Please use either resource principal or your API Key (config and private key) for authentication. When using the config, make sure to store the config and private key in the /home/datascience/.oci directory
- Please make sure all policies for both the notebook (dynamic group) and personal policies are in place to build, store, and deploy a model

# **Step 1 - Install and Publish Custom Conda environment**

To deploy the re-ranker model, this example uses FlagEmbedding. However, FlagEmbedding is not by default installed in the Conda environments. So, you have to install FlagEmbedding in the Conda environment and publish the (then custom) conda environment. That way, when you deploy the re-ranker model, the runtime environment (conda) used includes the FlagEmbedding package.

## Step 1.1 - Create a Custom Conda environment

Follow the described steps [here](https://docs.oracle.com/en-us/iaas/data-science/using/conda_create_conda_env.htm) to create a custom Conda environment

## Step 1.2 - Install dependencies

After installing the custom conda environment, please make sure the following packages are installed in the custom conda environment ("pip install [package]")
- oracle-ads --upgrade
- pandas
- json
- FlagEmbedding

## Step 1.3 - Run the below imports

In [ ]:
import ads
import pandas as pd
import json
from FlagEmbedding import FlagReranker
from ads.model.generic_model import GenericModel
from ads.model.model_metadata import MetadataCustomCategory

## Step 1.4 - Publish the custom Conda environment

Follow the described steps [here](https://docs.oracle.com/en-us/iaas/data-science/using/conda_publishs_object.htm) to publish the custom Conda environment to object storage

In [ ]:
!odsc conda init -b [YOUR_BUCKET_NAME] -n [YOUR_NAMESPACE] -a resource_principal
!odsc conda publish -s [THE_NAME_OF_YOUR_CUSTOM_CONDA] --force

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

# **Step 2 - Build the Re-ranker model**

## Step 2.1 - Define the FlagReranker model

Run the below cell. This will define the FlagReranker model and in a later stage will pull the model artifact files in memory and build a pickle file

In [ ]:
class Reranker:
    def __init__(self, model_id):
        self.model_id = model_id
        self.reranker = FlagReranker(self.model_id, use_fp16=True)

    def predict(self, x):
        scores = self.reranker.compute_score(x)

        return scores
    
model = Reranker(model_id="BAAI/bge-reranker-large")

## Step 2.2 - Build the Model Artifacts

In the below cell, add your personal Bucket name and Namespace. The below refers to the full location of the published custom conda environment, which you can find in your object storage. In the example below, "Tensorflow 2.8 for GPU Python 3.8" was used. However, please change the full location to your personal custom conda environment.

In [ ]:
custom_conda = "oci://[YOUR_BUCKET]@[YOUR_NAMESPACE]/conda_environments/gpu/TensorFlow 2.8 for GPU on Python 3.8/1.0/tensorflow28_p38_gpu_v1"

Run the below cell. This will build a new folder named "reranker_dir, including a score.py, a runtime.yaml and the reranker model as pickle file

In [ ]:
## Define the model
reranker_model = GenericModel(estimator=model, artifact_dir="./reranker_dir")

## Build artifact files
reranker_model.prepare(
    reload=False,
    inference_conda_env=custom_conda,
    inference_python_version="3.9",
    model_file_name="reranker.pkl",
    force_overwrite=True,
)

## Step 2.3 - Alter the score.py

To customize the output of the model, we are manually changing the score.py, which loads and runs the model on the provided input and provides the output

In [ ]:
%%writefile ./reranker_dir/score.py

import json
import os
import cloudpickle
import pandas as pd
import numpy as np
from functools import lru_cache


def load_model():
    class DummyModel:
        def __init__(self):
            pass
    return DummyModel()


def predict(data, model=load_model()):
    
    model_name = 'reranker.pkl'
    #model_name = './reranker_dir/reranker.pkl'
    
    with open(model_name, "rb") as file:
        model = cloudpickle.load(file)

    payload = data
    
    #get the origial question
    original_question = payload['full_input']['original_question']
    
        ##create payload as expected by rerank model

    payload_list = []
    

    for rowx in payload['full_input']['result']:

        text = rowx['payload']['text']

        payload_list.append([original_question, text])

    #apply list to rerank model
    rerank_predictions = model.predict(payload_list)

    loopx = 0

    #create output list
    output_list = []
    for rowyy in rerank_predictions:

        #get the text
        output_text = payload['full_input']['result'][loopx]['payload']['text']

        #add score
        output_list.append([output_text, rowyy])

        loopx+=1

    ########################
    #load as dataframe and sort on score
    df = pd.DataFrame(output_list, columns=["text","score"])
    df = df.sort_values('score', ascending=False)

    df_to_json = df.to_json(orient = 'records')
    
    output = json.loads(df_to_json)
    
    return {'prediction':output}

## Step 2.4 - Review the artifacts

Run the below cell. This reviews the artifact files and notifies you when there is something missing or wrong

In [ ]:
reranker_model.introspect()

# **Step 3 - Store the model in the model catalog**

In the below cell, please change the bucket name and namespace. When running the cell, this will store the full reranker_dir in the Model Catalog in OCI Data Science. However, the pickle file used is relatively large, so we are using the object storage to help in storing the model in the model catalog. When you run the cell, the model appears in the Model Catalog

In [ ]:
# Saving the model artifact to the model catalog. 
catalog_entry = reranker_model.save(display_name='rerank_model_v1', description='rerank_model_v1', timeout=600, bucket_uri="oci://[YOUR_BUCKET]@[YOUR_NAMESPACE/")
catalog_entry.id

# **Step 4 - Deploy the model**

Follow the described steps [here](https://docs.oracle.com/en-us/iaas/data-science/using/model-dep-create-def-net.htm#model-dep-create-def-net) to select and deploy the model in Model Deployment using the OCI Console


# **Step 5 - Invoke the model**

## Step 5.1 - Create example payload

The below is an example payload of the output of a vector embedding search. There is an "original question" and the top 5 embeddings (as text) returned from the vector database. The re-ranker model will use the original question and the "text" to re-rerank the results

In [15]:
payload = {'full_input': {'original_question': 'When do you go home?',
  'result': [{'id': 5469,
    'version': 1,
    'score': 0.7420407,
    'payload': {'creation_date': '2024-01-29Z',
     'document_name': 'autonomous-database-self-repairing-5116047 (1).pdf',
     'hyperlink_url': '',
     'page_number': 3,
     'text': '\nTable of Contents\nIntroduction\n4\nWhat is an Autonomous Database?\n4',
     'type': 'pdf'},
    'vector': None},
   {'id': 5483,
    'version': 2,
    'score': 0.7236079,
    'payload': {'creation_date': '2024-01-29Z',
     'document_name': 'autonomous-database-self-repairing-5116047 (1).pdf',
     'hyperlink_url': '',
     'page_number': 5,
     'text': '\nor eliminate operational disruption.\nWhat is the Autonomous Database Cloud?\nAUTONOMOUS\nDATABASE\nORACLE\n-',
     'type': 'pdf'},
    'vector': None},
   {'id': 5542,
    'version': 5,
    'score': 0.67869973,
    'payload': {'creation_date': '2024-01-29Z',
     'document_name': 'autonomous-database-self-repairing-5116047 (1).pdf',
     'hyperlink_url': '',
     'page_number': 12,
     'text': '\nNo databases that run on-premises or in cloud environments today are 100% autonomous - but that is\nthe goal toward which the industry is headed. To further the evolution of cloud databases toward this\ntrue utility model, Oracle introduced the Autonomous Database, running on Oracle Database (version\n18c and later) in the Oracle Cloud. Autonomous Database minimizes or eliminates human labor using\nself-driving, self-securing and self-repairing functionality. Two key areas that comprise the self-\nrepairing capabilities of the Autonomous Database are the Oracle Maximum Availability Architecture',
     'type': 'pdf'},
    'vector': None},
   {'id': 5496,
    'version': 2,
    'score': 0.67500746,
    'payload': {'creation_date': '2024-01-29Z',
     'document_name': 'autonomous-database-self-repairing-5116047 (1).pdf',
     'hyperlink_url': '',
     'page_number': 6,
     'text': "\nbehind corporate firewalls to meet data sovereignty or control requirements will soon be able to run\nthe Autonomous Database on-premises. Oracle Exadata Cloud at Customer, an Oracle Public Cloud\noffering, can be deployed on-premises, and delivers all of the capabilities of Autonomous Database\nfrom within the enterprise's data center.\nThe Autonomous Database can be deployed in a hybrid cloud or all-cloud model; for example, when\nmultiple databases are deployed for production and test environments or as primary and standby",
     'type': 'pdf'},
    'vector': None},
   {'id': 5476,
    'version': 1,
    'score': 0.6721611,
    'payload': {'creation_date': '2024-01-29Z',
     'document_name': 'autonomous-database-self-repairing-5116047 (1).pdf',
     'hyperlink_url': '',
     'page_number': 4,
     'text': '\nif you will. As a result, enterprises are unable to realize the full operational and financial benefits of the\ncloud.\nWHAT IS AN AUTONOMOUS DATABASE?\nThere is understandably an element of confusion that arises when talking about automatic" versus\n\'autonomous" capabilities. A process for database backup, failover or resizing that can be\naccomplished automatically is still not autonomous if a database administrator has to respond to an',
     'type': 'pdf'},
    'vector': None}],
  'status': 'ok',
  'time': 0.00130104}}

## Step 5.2 - Invoke the Model Deployment HTTP endpoint

In the below cell, add the Model Deployment's HTTP Endpoint. When you run the cell, the HTTP endpoint will be invoked using the example payload. The config file is used as authentication.

In [ ]:
import requests
import oci
from oci.signer import Signer
import json

your_http_endpoint = f""


config = oci.config.from_file("~/.oci/config") # replace with the location of your oci config file
auth = Signer(
        tenancy=config['tenancy'],
        user=config['user'],
        fingerprint=config['fingerprint'],
        private_key_file_location=config['key_file'],
        pass_phrase=config['pass_phrase'])


#POST request to the model
response = requests.post(your_http_endpoint, json=payload, auth=auth)
response